## Добрый день, дорогие друзья!

Код для статьи портала [NTA](https:\\newtechaudit.ru).   
Оптимизация рутинных задач в машинном обучении.  

Мы обучим три модели - LGBM с ручной настройкой параметров, градиентный буст с оптимизированными параметрами с применением optuna и результат работы фреймворка h2o.automl. Алгоритм работы featuretools представлен в качестве базового примера автоматизации инжиниринга новых фич.

## Подготовка библиотек и данных

Загружаем классические библиотеки и данные

In [ ]:
import numpy as np 
import pandas as pd 
import math, random

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', 100)

from lightgbm import LGBMRegressor

SEED = 47

In [ ]:
PATH = '../input/tabular-playground-series-jan-2021/'

df_train = pd.read_csv(PATH + '/train.csv')
df_test = pd.read_csv(PATH + '/test.csv')
df_sub = pd.read_csv(PATH + '/sample_submission.csv')

In [ ]:
target = df_train['target']
features = df_train.drop('target', axis=1)

In [ ]:
features.head()

## Featuretools

In [ ]:
# Загрузка фреймворка
import featuretools as ft

In [ ]:
# Создаем новую сущность - EntitySet

es = ft.EntitySet(id = 'data')
es.entity_from_dataframe(entity_id = 'january', 
                         dataframe = features, 
                         index='id')
                         #make_index = True, index = 'index') # Эта строка поможет создать новый индекс, при его отсутсвии

# Запускаем создание новых признаков
feature_matrix, feature_defs = ft.dfs(entityset = es,                                          # Какой EntiteSet обрабатываем
                                      target_entity = 'january',                               # Какой датафрейм изменяем
                                      trans_primitives = ['add_numeric', 'multiply_numeric'],  # Какие фичи создаем
                                      verbose=1)                                               # Показывать ли прогресс выполнения

In [ ]:
# Обновленный датайфрем содержит сгенерированные признаки
feature_matrix.head()

Мы не будем обучать модель на полученных данных, так как предполагаем сильную мультиколлениарность. Пример приведен в качестве учебного и для знакомства с фреймворком.

## Optuna

In [ ]:
# Подгружаем фреймворк и встроенные средства визуализации результатов

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

In [ ]:
# В учебном примере некоторые гиперпараметры выключены из пространства поиска с целью ускорения процесса. 
# Для более глубокого подбора, раскомментируйте интересующие строки.

def objective(trial, data=features, target=target):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
    
    params = {
        'metric': 'rmse', 
        'random_state': SEED,
        'n_estimators': 10000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        #'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.75,0.8,0.85]),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.65,0.7,0.75,0.8,0.85]),
        'learning_rate': trial.suggest_categorical('learning_rate', 
                                                   [0.005,0.006,0.008,0.01,0.015,0.02,0.03]),
        #'max_depth': trial.suggest_categorical('max_depth', [-1,10,20]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 300),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        #'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    
    # Создаем и обучаем модель с мониторингом переобучения
    model = LGBMRegressor(**params)      
    model.fit(X_train, y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=300, verbose=False)
    preds = model.predict(X_test)
    
    #Смотрим финальную метрику RMSE
    rmse = mean_squared_error(y_test, preds, squared=False)
    
    return rmse

In [ ]:
%%time
# Посчитаем время оптимизации

# Создаем задание для фреймворка
study = optuna.create_study(direction='minimize')  # Минимизируем ошибку
study.optimize(objective, n_trials=5)              # Количество итераций = 5

# Смотрим на финальные метрики
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

In [ ]:
# История оптимизации
plot_optimization_history(study)

In [ ]:
# Значимость гиперпараметров при настройке
plot_param_importances(study)

In [ ]:
# Выводим лучшие гиперпараметры
study.best_params

Создаем словарь с обновленными параметрами, обучаем модель и создаем файл для сабмита. В новый словарь необходимо добавить постоянные значения, такие, как random_state, исходная метрика и количество деревьев.

In [ ]:
optuna_params = study.best_params

optuna_params['metric'] = 'rmse'
optuna_params['random_state'] = SEED
optuna_params['n_estimators'] = 10000

In [ ]:
# Выделяем валидационный сет из обучающего набора данных
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=SEED)

# Создаем и обучаем модель с подобранными гиперпараметрами
model_optuna = LGBMRegressor(**optuna_params)
model_optuna.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=300, verbose=300)

In [ ]:
# Cоздаем предсказание и файл для сабмита

pred_optuna = model_optuna.predict(df_test)

df_sub['target'] = pred_optuna
df_sub.to_csv('submission_optuna.csv', index=False)

In [ ]:
df_sub.head()

## AutoML

**В этом разделе мы сравним классический процесс обучения модели и работу automl.**


Обучим LGBM с простой ручной настройкой гиперпараметров на сырых данных. В конце сравним этот результат с результатом работы AutoML. Применяем стандартный процесс обучения и предикта без кросс-валидации и отложенных семплов. 

In [ ]:
# Создаем модель с небольшой ручной настройкой гиперпараметров

model_default = LGBMRegressor(n_estimators=10000,
                      max_depth=-1, 
                      reg_alpha=2,
                      reg_lambda=1.5,
                      num_leaves=37,
                      metric='rmse',
                      random_state=SEED)

# Обучаем созданную модель с мониторингом переобучения
model_default.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=300, verbose=300)

In [ ]:
# Cоздаем предсказание и файл для сабмита

pred_default = model_default.predict(df_test)

df_sub['target'] = pred_default
df_sub.to_csv('submission_default.csv', index=False)

In [ ]:
df_sub.head()

Очередь AutoML

In [ ]:
# Загружаем фреймворк и устанавливаем максимальный размер используемой оперативной памяти

import h2o
print(h2o.__version__)

from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

In [ ]:
%%time
train = h2o.import_file("../input/tabular-playground-series-jan-2021/train.csv")
test = h2o.import_file("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
x = test.columns[1:]
y = 'target'

In [ ]:
# Запускаем автоматическое обучение на трех моделях
aml = H2OAutoML(max_models=2, 
                seed=SEED, 
                max_runtime_secs=31000)

aml.train(x=x, y=y, training_frame=train)

In [ ]:
# Смотрим на лучшие модели

lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
# Изучаем модель - лидера

aml.leader

In [ ]:
# Создаем предсказания и отправляем сабмит

preds = aml.predict(test)

df_sub['target'] = preds.as_data_frame().values.flatten()
df_sub.to_csv('submission_automl.csv', index=False)